# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [3]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [4]:
import numpy as np
import copy

class Sudoku_Sate:

    def __init__(self, original_grid,possibleValues, unknown_positions):
        self.grid = copy.deepcopy(original_grid)
        dimensions = self.grid.shape
        self.gh = dimensions[0]
        self.gw = dimensions[1]

        #only recalculates the grid if None is passed, otherwise it will use the same values that are passed in
        if possibleValues == None:
            static_possibleVals = np.empty([self.gh, self.gw, 9])
            static_possibleVals.fill(-1)
            self.possibleVals = static_possibleVals.tolist()
            self.unknown_pos = []
            self.calc_AllPossibles()
        else:
            self.possibleVals = copy.deepcopy(possibleValues)
            self.unknown_pos = copy.deepcopy(unknown_positions)

    #calcultes all the possible values for a given grid
    def calc_AllPossibles(self):
        for i in range(self.gh):
            for j in range(self.gw):
                if self.grid[i][j] == 0:
                    self.possibleVals[i][j] = self.calc_PossibleVals(i, j, self.grid)
                    self.unknown_pos.append([i,j])
                else:
                    self.possibleVals[i][j] = []

    #calcultes the possible values after setting a new value,
    #so only alters the row col and square it affceted
    def calc_PossibleVals(self, x,y,grid):
        p = [1,2,3,4,5,6,7,8,9]
        #removing from the row
        for j in range(self.gw):
            if grid[x][j] in p:
                p.remove(grid[x][j])
        #removing from the column
        for i in range(self.gh):
            if grid[i][y] in p:
                p.remove(grid[i][y])

        s = 3 #sqaure size, 3 by 3 for a 9x9
        x_offset = s * (x//s)
        y_offset = s * (y // s)
        for i in range(s):
            for j in range(s):
                if grid[i+x_offset][j+y_offset] in p:
                    p.remove(grid[i+x_offset][j+y_offset])
        return p

    #once has set a value this is called to tidy up the possible values
    def remove_possibles(self,x,y,val):
        for j in range(self.gw):
            if val in self.possibleVals[x][j]:
                self.possibleVals[x][j].remove(val)
        for i in range(self.gh):
            if val in self.possibleVals[i][y]:
                self.possibleVals[i][y].remove(val)
        #square ise = s
        s = 3
        x_offset = s * (x//s)
        y_offset = s * (y // s)
        for i in range(s):
            for j in range(s):
                if val in self.possibleVals[i+x_offset][j+y_offset]:
                    self.possibleVals[i + x_offset][j + y_offset].remove(val)

    #automatiocaly assigns any values that only have one possible value
    def fix_Singletons(self):
        for i in range(self.gh):
            for j in range(self.gw):
                if len(self.possibleVals[i][j]) == 1:
                    self.set_value(i,j,self.possibleVals[i][j][0])

    #sets the value in the grid based on its possible values
    def set_value(self, i, j, val):
        #assigning the value
        self.grid[i][j] = val
        self.possibleVals[i][j] = []
        try:
            self.unknown_pos.remove([i,j])
        except:
            print("You just set a value that wasn't unknonw... need to fix code")

        self.remove_possibles(i,j,val)

    #call this when want to set a value AND sort out any singletons
    def set_value2(self, x, y, val):
        self.set_value(x,y,val)
        self.fix_Singletons()

    #checks if the state is invalid or not
    def is_invalid(self):
        for pos in self.unknown_pos:
            if len(self.possibleVals[pos[0]][pos[1]]) == 0:
                return True
        return False

    #checks the grid if it contains the value x
    def gridContains(self, x):
        for i in range(self.gh):
            if x in self.grid[i]:
                return True
        return False

    #checks if you have every number in every row
    def correctGrid(self):
        nums = np.arange(1,9)
        nums = nums.tolist()
        for i in range(self.gh):
            for j in range(self.gw):
                if self.grid[i][j] in nums:
                    nums.remove(self.grid[i][j])
            #checks each row to make sure that it only has one of each value, if not then not a corrrect grid
            if len(nums) == 0:
                nums = np.arange(1,9)
                nums = nums.tolist()
            else:
                return False
        return True

    #checks if grid is solved
    def is_goal(self):
        if(len(self.unknown_pos) == 0) and (self.gridContains(0)==False):
            if self.correctGrid():
                return True
        else:
            return False

    #getters-----
    def get_possibleVals(self, x,y):
        return self.possibleVals[x][y]

    def get_grid(self):
        return self.grid


#---------------------------DETH FIRST SEARCH-------------------------------------------

#finds the next position to try on the grid with the least number of unknonw values
def find_next_unknown(hieght,width, poss_Vs):
    smallest = 9
    pos = [-1,-1]
    for n in range(hieght):
        for m in range(width):
            if len(poss_Vs[n][m]) < smallest and len(poss_Vs[n][m])>0:
                smallest = len(poss_Vs[n][m])
                pos = [n,m]
    return pos

def depthFirst_Search (oldGrid, possibleValues, unknown_positions):
    #populates the current state object of sudoku depending on whether it has been passed possible values or not
    currState = Sudoku_Sate(oldGrid, possibleValues, unknown_positions)
    #finds the position to try to set the value of
    position = find_next_unknown(currState.gh, currState.gw, currState.possibleVals)
    if position == [-1,-1]:
        if not currState.is_invalid():
            return currState
        else:
            return None

    poss_vals = currState.get_possibleVals(position[0],position[1])
    for v in poss_vals:
        newState = Sudoku_Sate(currState.get_grid(), possibleValues, unknown_positions)
        newState.set_value2(int(position[0]), int(position[1]), v)

        if newState.is_goal():
            return newState
        if not newState.is_invalid():
            consequenceState = depthFirst_Search(newState.get_grid(), newState.possibleVals, newState.unknown_pos)
            if (consequenceState is not None) and (consequenceState.is_goal()):
                return consequenceState

    return None



def sudoku_solver(sudoku):
    solved_state = depthFirst_Search(sudoku, None, None)
    
    if solved_state != None:
        solved_sudoku = solved_state.get_grid()
    else:
        broke = [[-1] * 9 for _ in range(9)]
        solved_sudoku = np.asarray(broke)
    return solved_sudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [7]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5

This is easy sudoku number 0
[[8 5 2 9 7 6 2 4 3]
 [6 7 9 1 4 3 2 8 5]
 [0 3 1 2 5 8 7 6 9]
 [3 1 4 5 2 7 8 9 6]
 [7 6 8 3 9 1 4 5 0]
 [9 2 5 6 0 0 3 7 1]
 [5 4 3 8 6 2 9 1 7]
 [1 9 7 4 3 5 0 2 8]
 [2 8 6 7 1 9 5 3 4]]
This is your solution for easy sudoku number 0
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is easy sudoku number 1
[[3 7 4 5 8 0 6 2 9]
 [8 2 5 7 9 6 4 0 1]
 [6 9 1 2 3 4 8 7 5]
 [1 4 2 0 5 3 7 9 8]
 [7 8 6 9 4 2 1 5 3]
 [9 5 3 8 1 7 2 4 6]
 [4 6 9 1 7 5 3 8 2]
 [5 1 7 3 2 8 0 6 4]
 [2 3 8 6 0 9 5 1 7]]
This is your solution for easy sudoku number 1
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -

This sudoku took 0.0 seconds to solve.

15/15 easy sudokus correct
Testing medium sudokus
This is medium sudoku number 0
[[1 6 0 2 5 3 0 0 8]
 [3 5 2 0 7 8 6 1 4]
 [8 7 9 0 6 4 0 2 3]
 [6 2 3 7 0 0 8 4 9]
 [7 0 5 8 0 0 0 3 6]
 [9 4 8 3 2 5 1 0 0]
 [5 8 6 0 3 2 7 9 1]
 [0 3 7 6 9 1 0 0 5]
 [4 9 0 5 8 7 3 6 0]]
This is your solution for medium sudoku number 0
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.03125 seconds to solve.

This is medium sudoku number 1
[[5 0 9 0 3 8 0 2 7]
 [2 3 0 4 6 7 1 9 5]
 [6 7 4 0 1 0 8 0 3]
 [0 8 1 0 2 4 3 5 0]
 [9 2 0 1 0 0 7 6 4]
 [3 4 6 7 9 5 0 8 1]
 [0 0 7 0 4 2 0 1 6]
 [4 6 3 8 5 1 9 7 0]
 [0 5 2 9 7 6 4 3 8]]
This is your solution for medium sudoku number 1
[[-1 -

This is your solution for hard sudoku number 0
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.703125 seconds to solve.

This is hard sudoku number 1
[[1 0 0 7 0 0 0 0 0]
 [0 3 2 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0]
 [0 8 0 0 0 2 0 7 0]
 [5 0 7 0 0 1 0 0 0]
 [0 0 0 0 0 3 6 1 0]
 [7 0 0 0 0 0 2 0 9]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 0 0 4 0 0 5]]
This is your solution for hard sudoku number 1
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
Th

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [6]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a TEST CELL. Do not delete or change.